In [1]:
from collections.abc import Iterable
import numba as nb
import numpy as np
import matplotlib.pyplot as plt

### Оценка среднего расстояния, пройденного волной.

In [2]:
def _tus_from_args(storage, t_min, t_max, tus):
    if storage:
        tus = storage.load_tus(t_min=t_min, t_max=t_max)
    elif storage is None and tus is not None:
        idx = np.logical_and(tus[:, 0] >= t_min, tus[:, 0] <= t_max)
        tus = tus[idx]
    else:
        raise ValueError('Neither storage nor tus is specified.')
    return tus

In [3]:
@nb.njit('f8(f8[:, :])')
def integrate_trapeze(tus):
    res = 0.
    for i in range(1, tus.shape[0]):
        res += (tus[i, 1] + tus[i - 1, 1]) * (tus[i, 0] - tus[i - 1, 0])
    res *= 0.5
    return res

In [4]:
@nb.njit
def _find_minima_idx(tus):
    idx = []
    i = 0
    while i < tus.shape[0] - 2:
        i += 1
        while not (tus[i - 1, 1] > tus[i, 1] < tus[i + 1, 1]) and i + 1 < tus.shape[0] - 1:
            i += 1
        idx.append(i)
    if idx:
        idx.pop()
    return np.array(idx)

In [ ]:
@nb.njit
def _find_maxima_idx(tus):
    idx = []
    i = 0
    while i < tus.shape[0] - 2:
        i += 1
        while not (tus[i - 1, 1] < tus[i, 1] > tus[i + 1, 1]) and i + 1 < tus.shape[0] - 1:
            i += 1
        idx.append(i)
    if idx:
        idx.pop()
    return np.array(idx)

In [5]:
def find_minima(storage=None, t_min=1000, t_max=np.inf,
                plot=False, ret_idx=False, main=False, tus=None):
    '''Finds points of tus with minimal us'''
    tus = _tus_from_args(storage, t_min, t_max, tus)
    idx = _find_minima_idx(tus)
    minima = tus[idx, :]
    if main:
        ptp = minima[minima[:, 0] >= 1000, 1].ptp()
        if ptp > 5e-4:
            while True:
                minima_next, idx_next = \
                    find_minima(tus=minima, t_min=t_min, t_max=t_max, ret_idx=True)
                if np.unique(minima_next[:, 1]).size < 2 or ptp < 5e-6:
                    break
                minima, idx = minima_next, idx[idx_next]
                ptp = minima_next[:, 1].ptp()
    if plot:
        plt.plot(tus[:, 0], tus[:, 1])
        plt.gcf().set_size_inches((10, 5))
        plt.grid()
        plt.minorticks_on()
        plt.grid(which='minor', linestyle=':', linewidth='0.5', color='grey')
        plt.scatter(minima[:, 0], minima[:, 1], c='red', marker='x')
    if ret_idx:
        return minima, idx
    return minima

In [ ]:
def find_maxima(storage=None, t_min=1000, t_max=np.inf,
                plot=False, ret_idx=False, main=False, tus=None):
    '''Finds points of tus with minimal us'''
    tus = _tus_from_args(storage, t_min, t_max, tus)
    idx = _find_maxima_idx(tus)
    maxima = tus[idx, :]
    if main:
        ptp = maxima[maxima[:, 0] >= 1000, 1].ptp()
        if ptp > 5e-4:
            while True:
                maxima_next, idx_next = \
                    find_maxima(tus=maxima, t_min=t_min, t_max=t_max, ret_idx=True)
                if np.unique(maxima_next[:, 1]).size < 2 or ptp < 5e-6:
                    break
                maxima, idx = maxima_next, idx[idx_next]
                ptp = maxima_next[:, 1].ptp()
    if plot:
        plt.plot(tus[:, 0], tus[:, 1])
        plt.gcf().set_size_inches((10, 5))
        plt.grid()
        plt.minorticks_on()
        plt.grid(which='minor', linestyle=':', linewidth='0.5', color='grey')
        plt.scatter(maxima[:, 0], maxima[:, 1], c='red', marker='x')
    if ret_idx:
        return maxima, idx
    return maxima